# PDE Modelling Mimic of 190413 experiment

## Objectives 
* Microscope movie was performed on 190413 that simply observed growth of a constitutively fluorescent strain on agar pads, varying in pad size and cell occupation
* Simulate each of the experimental setups executed
* eventually build into a fitting routine

## Model considerations 
* Species
    1. Cell density (sender and pulse cells)
    1. Nutrient density 
    1. mScarlet
* Reactions 
    1. cell growth and diffusion 
        * Cells diffuse very slowly
        * nutrient-dependent growth (from Liu et al 2011, Science) 
        $$  $$ 
    1. Constitutive fluorescence
        * Basal protein expression 
        * initial protein concentration set to fixed point of max nutrient
    1. Dilution and degradation 
        * Assume that all proteins are degradation tagged
    1. Diffusion 
        * Here, you're going to use convoultion of the diffusion kernel
        * Diffusion in/out of cell is considered faster than spatial diffusion at these scales
    1. Parameters
        * We are also assuming, for the moment, that each time point is 6 minutes. Parameters with time dimensions shown below may use different units than the parameter from the cited paper.
        * dx: Length modification of diffusion terms. In the compartmental model, diffusion is calculated via Ficks' first law, where the flux between two adjacent compartments is equal to the flux multiplied by the area of the interface between the components :  
        $\frac{\mathrm{d} C}{\mathrm{d} t} $ 
        in continuous form gives up 
        $\Delta C = D \frac{A}{V} \frac{\Delta C}{\Delta x} = D \frac{2.25 \cdot 5 \cdot \mathrm{scale}^2 \mathrm{mm}^2}{\mathrm{scale} \cdot 2.25^2 \cdot 5 \mathrm{mm}^3} \frac{\Delta C \cdot \mathrm{scale}}{2.25 \mathrm{mm}} = \frac{D \Delta C \mathrm{scale}^2}{2.25^2 \mathrm{mm}^2}$. the dx parameter below is the symbol $A$ in this equation.
        * Dc : Diffusion rate for cells. $7\frac{mm^2}{min}$
        * rc : Division rate of cells. $\frac{1.14}{min}$
        * Kn : Half-point of nutrient availability. 75
        * Dn : Diffusion rate of nutrient. $28\frac{mm^2}{min}$
        * kn : Consumption rate of nutrient by cells
        * Da : Diffusion rate of nutrient. $28\frac{mm^2}{min}$
        * xs : Expression rate of protein. 


In [13]:
# imports
from __future__ import division, print_function

import itertools as itt

import numpy as np
import pandas as pd
import os
import sys
import string
import selenium
import scipy.integrate as itg
import scipy.optimize as opt
import scipy.interpolate as itp
import scipy.ndimage as ndi
import scipy.sparse as sparse

import matplotlib as mpl
mpl.use("Agg")

import itertools

import matplotlib.pyplot as plt 
# plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg' # Add the path of ffmpeg here!!

import matplotlib.animation as anm
import skimage.measure
import skimage.filters
import numba
import gc

from multiprocessing import Pool, Process

%load_ext memory_profiler
%load_ext line_profiler
import timeit

# import bokeh
# from bokeh.plotting import figure, output_file, save
# from bokeh.io import output_notebook, show
# from bokeh import palettes, transform
# from bokeh.models import LogColorMapper, LogTicker, ColorBar, LinearColorMapper, Ticker
# output_notebook()

from IPython.display import HTML

# rc = {'lines.linewidth': 2, 
#       'axes.labelsize': 18, 
#       'axes.titlesize': 24, 
#       'xtick.labelsize': 18, 
#       'ytick.labelsize': 18, 
#       'legend.fontsize': 18,
#       'axes.facecolor': 'DFDFE5'}

# sns.set_context('paper', rc=rc)

%matplotlib inline

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


## 2D Discrete Laplacian

In continuous form : 
$$ U_t = \triangle U - \lambda U $$

In discrete form, for point $i$ : 
$$ \Delta U_i = \sum_{1 = w(i,j)}\omega(i,j)(U_i - U_j) - \lambda U_i $$

Use discrete laplacian approximation w/o diagonals for grid spacing, so that we can have zero-flux  boundary conditions. 

$$ L = 
 \begin{pmatrix}
  0 & 1 & 0 \\
  1 & -4 & 1 \\
  0 & 1 & 0 
 \end{pmatrix} $$

I use a convolution function to calculate the diffusion terms. 

# Helper functions used to define the arenas 
### Needs
* read excel or csv files 
* rescaling arrays and contents 
* convert row/col to array index


* disk function, projects circular areas onto an input grid 
* 

In [2]:
#@numba.jit('void(float32[:,:,:],float32[:,:,:])', nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def calc_diffusion(A, D):
    # Middle
    D[:,1:-1,1:-1] = A[:,1:-1, 2:] + A[:,1:-1, :-2] + A[:,:-2, 1:-1] + A[:,2:, 1:-1] - 4*A[:,1:-1, 1:-1]
    # Edges
    D[:,0,1:-1] = A[:,0, 2:] + A[:,0, :-2] + A[:,1, 1:-1] - 3*A[:,0, 1:-1]
    D[:,-1,1:-1] = A[:,-1, 2:] + A[:,-1, :-2] + A[:,-2, 1:-1] - 3*A[:,-1, 1:-1]
    D[:,1:-1,0] = A[:,2:,0] + A[:,:-2,0] + A[:,1:-1,1] - 3*A[:,1:-1,0]
    D[:,1:-1,-1] = A[:,2:,-1] + A[:,:-2,-1] + A[:,1:-1,-2] - 3*A[:,1:-1,-1]
    # Corners
    D[:,0,0] = A[:,0,1] + A[:,1,0] - 2*A[:,0,0]
    D[:,-1,0] = A[:,-1,1] + A[:,-2,0] - 2*A[:,-1,0]
    D[:,0,-1] = A[:,0,-2] + A[:,1,-1] - 2*A[:,0,-1]
    D[:,-1,-1] = A[:,-1,-2] + A[:,-2,-1] - 2*A[:,-1,-1]

#@numba.jit('float32[:,:](float32[:,:],float32,float32)',nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def hill(a, n, k):
    h_ma = 1 - (1 / (1 + (a/k)**n))
    return h_ma

@numba.jit(nopython=True, cache=True)
def dhillda(a, n, k):
    h_ma = (n/k)*((a/k)**(n-1))*(1 / (1 + (a/k)**n)**2)
    return h_ma

#@numba.jit('float32[:,:](float32[:,:],float32,float32)',nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def hillN(a, n, k):
    return 1 / (1 + (a/k)**n)

# @numba.jit(cache=True)
def f_ivp(t, y, d_y, diff_terms, nut_avail, p0, dims, calc_f):
    species, n_h, n_w, scale= dims
    y.shape = (species, n_h, n_w)
    calc_f(y, d_y, diff_terms, nut_avail, p0)
    y.shape = species*n_h*n_w
    return d_y.flatten()

solver = 'RK45'

def wrapper(dims, p0, initial_array, tmax, atol, rtol, calc_f, jac):
    print(solver)
    species, n_h, n_w, scale = dims
    args=(np.zeros((species, n_h, n_w), dtype=np.float32,order='C'), 
          np.zeros((species, n_h, n_w), dtype=np.float32,order='C'), 
          np.zeros((n_h, n_w), dtype=np.float32,order='C'), 
          p0, dims, calc_f)
    initial_array.shape = n_h*n_w*species
    f_lambda = lambda t, y : f_ivp(t, y, *args)
    out = itg.solve_ivp(f_lambda, [0, tmax], initial_array.copy(), vectorized=True, method=solver, 
                        atol=atol, rtol=rtol, t_eval=np.arange(0,tmax,4), jac=jac)
    exp_t = out.t
    exp_y = out.y.T
    exp_y.shape = (len(exp_t), species, n_h, n_w)
    return exp_y, exp_t

# @numba.jit(cache=True)
def prep_pad_helper(scale, init_cells):
    scale_factor = (scale/4500)/(2.475/4)
    scaled_init = skimage.transform.rescale((init_cells>1).astype(np.float32), 
                                            scale_factor, 
                                            order=1, 
                                            mode='constant', 
                                            multichannel=False,
                                            cval=0)
    return scaled_init

# @numba.jit(cache=True)
def prep_pad_0(scale):
    n_h, n_w = (2*scale,2*scale)
    cell_init = np.zeros((n_h, n_w), dtype=np.float32)
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_0_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = (np.array((n_h, n_w))//2) - np.array((s_h, s_w))//2
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    return cell_init

# @numba.jit(cache=True)
def prep_pad_1(scale):
    n_h, n_w = (2*scale,2*scale)
    cell_init = np.zeros((n_h, n_w), dtype=np.float32)
    # middle colony
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_1_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = (np.array((n_h, n_w))//2) - np.array((s_h, s_w))//2
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    # top colony
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_10_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = (np.array((0, n_w-s_w))//2)
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    return cell_init

# @numba.jit(cache=True)
def prep_pad_2(scale):
    n_h, n_w = (2*scale,2*scale)
    cell_init = np.zeros((n_h, n_w), dtype=np.float32)
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_2_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = (np.array((n_h, n_w))//2) - np.array((s_h, s_w))//2
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    return cell_init

# @numba.jit(cache=True)
def prep_pad_3(scale):
    n_h, n_w = (3*scale,3*scale)
    cell_init = np.zeros((n_h, n_w), dtype=np.float32)
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_3_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    # first colony
    y0, x0 = np.array((n_h//2 - s_h//2, n_w//8))
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    # duplicate on the opposite side for the second colony
    y0, x0 = np.array((n_h//2 - s_h//2, 6*n_w//8))
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    return cell_init

# @numba.jit(cache=True)
def prep_pad_4(scale):
    n_h, n_w = (3*scale,3*scale)
    cell_init = np.zeros((n_h, n_w), dtype=np.float32)
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_4_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = (np.array((n_h, n_w))//2) - np.array((s_h, s_w))//2
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    return cell_init

# @numba.jit(cache=True)
def prep_pad_5(scale):
    n_h, n_w = (5*scale,5*scale)
    cell_init = np.zeros((n_h, n_w), dtype=np.float32)
    # left colony
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_5_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = np.array(((n_h-s_h)//2, scale//8))
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    # right colony
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_6_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = np.array(((n_h-s_h)//2, 6*n_w//8))
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    return cell_init


# @numba.jit(cache=True)
def prep_pad_6(scale):
    n_h, n_w = (5*scale,5*scale)
    cell_init = np.zeros((n_h, n_w), dtype=np.float32)
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_7_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = np.array(((n_h-s_h)//2, scale//2))
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    return cell_init

# @numba.jit(cache=True)
def prep_pad_7(scale):
    n_h, n_w = (3*scale,3*scale)
    cell_init = np.zeros((n_h, n_w), dtype=np.float32)
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_8_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = np.array(((n_h-s_h)//2, (n_w-s_w)//2))
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    return cell_init
    
# @numba.jit(cache=True)
def prep_pad_8(scale):
    n_h, n_w = (2*scale,2*scale)
    cell_init = np.zeros((n_h, n_w), dtype=np.float32)
    img_fn = '../../../microscope-movie-analysis/worker_outputs/masks/pad_9_frame_5.tif'
    init_cells = skimage.io.imread(img_fn)
    scaled_init = prep_pad_helper(scale, init_cells)
    del init_cells
    s_h, s_w = scaled_init.shape
    y0, x0 = np.array(((n_h-s_h)//2, (n_w-s_w)//2))
    y1, x1 = np.array((y0,x0)) + np.array((s_h, s_w))
    y0, y1, x0, x1 = np.array((y0, y1, x0, x1), dtype=np.int)
    cell_init[y0:y1,x0:x1] = scaled_init
    return cell_init

# try to make a jacobian function
# you'll likely need to use the vectorized version of the ODE, rather than the matrix version. 
# first, just see if you can use the sparsity structure matrix approach
def sim_pad_prep(p0, scale, tmax, prep_fn):
    
    # Calculate dx and redefine p0
    Dc, Dn, rc, Kn, Hn, pn, xs, ps, leak, od0 = p0
    dx = np.power((scale/4.5),2)
    p0 = np.array([dx, Dc, Dn, rc, Kn, Hn, pn, xs, ps, leak, od0])
    
    # Prep initial array
    print('load initial cell frame')
    cell_init = prep_fn(scale)
    n_h, n_w = cell_init.shape
    col_thresh = 0.05
    species = 3 # cells, nutrients, mscarlet
    dims = [species, n_h, n_w, scale]
    c_i, n_i, s_i = np.arange(species)
    
    # Make empty array, and tolerance arrays
    print('make tolerance arrays')
    atol = np.zeros((species, n_h, n_w), dtype=np.float32,order='C')# + 1e-7
    atol[c_i,:,:] = 1e-3*np.ones((n_h, n_w), dtype=np.float32)
    atol[n_i,:,:] = 1e-2*np.ones((n_h, n_w), dtype=np.float32)
    atol[s_i,:,:] = 1e-2*np.ones((n_h, n_w), dtype=np.float32)
    # atol must be a vector for the solver
    atol.shape = species*n_h*n_w
    rtol = 1e-3
    
    # Set initial conditions
    initial_array = np.zeros((species, n_h, n_w), dtype=np.float32, order='C')# + 1e-7
    initial_array[n_i,:,:] = 100*np.ones((n_h, n_w), dtype=np.float32)
    initial_array[c_i,:,:] = cell_init*od0
    initial_array[s_i,:,:] = np.greater(cell_init, col_thresh)*(xs/(rc+ps))
    
    # Make jacobian array
    print('make jacobian array')
    dif_jac = 
    f_ji = lambda x, y, species : x + n_w*y + n_w*n_h*species

    # -4 center
    for x1 in np.arange(1,n_w-1):
        for y1 in np.arange(1,n_h-1):
            for species in np.arange(species):
                
    dif_jac_df.loc[sub_df.index,'jval'] -= 4*dx
    
    # correct for edges and corners
    x_mask = (sub_df.x1==0)|(sub_df.x1==(n_w-1))
    y_mask = (sub_df.y1==0)|(sub_df.y1==(n_h-1))
    sub_df.loc[x_mask|y_mask,'jval'] += dx
    sub_df.loc[x_mask&y_mask,'jval'] += dx
    
    # calculate diffusion from neighbors
    # sub_df = dif_jac_df.loc[s_mask,:]
    for xi, xj, yi, yj in [(1,0,0,0),(0,1,0,0),(0,0,1,0),(0,0,0,1)]:
        for flag in [1,-1]:
            x_mask = ((dif_jac_df.x1+flag*xi)==(dif_jac_df.x2+flag*xj))
            y_mask = ((dif_jac_df.y1+flag*yi)==(dif_jac_df.y2+flag*yj))
            idx = x_mask&y_mask&s_mask
            dif_jac_df.loc[idx,'jval'] += dx

    # Scale by diffusion rate
    sub_df = dif_jac_df.loc[s_mask,:]
    for spec_i, dc in [(c_i, Dc), (n_i, Dn), (s_i, 0)]:
        indx = sub_df.index[sub_df.species1==spec_i]
        dif_jac_df.loc[indx,'jval'] *= dc

#     xy_mask = (dif_jac_df.x1==dif_jac_df.x2)&(dif_jac_df.y1==dif_jac_df.y2)
#     sub_df = dif_jac_df.loc[xy_mask,:]
#     for s1,s2 in [(c_i,n_i),(n_i,c_i),(s_i,c_i),(s_i,n_i)]:
#         s_mask = (sub_df.species1==s1)&(sub_df.species2==s2)
#         dif_jac_df.loc[sub_df.loc[s_mask,:].index,'jval']=1
        
    dif_jac = dif_jac_df.pivot('j1','j2','jval').values
    xy_mask = (dif_jac_df.x1==dif_jac_df.x2)&(dif_jac_df.y1==dif_jac_df.y2)
    sub_df = dif_jac_df.loc[xy_mask,['x1','y1','j1','j2','species1','species2']].astype(np.int)
    rxn_jac = dif_jac.copy()
    
    #dc/(dcdt)
    sub_indx = sub_df.index[(sub_df.species1==c_i)&(sub_df.species2==c_i)]
    dcdcdt_indices = sub_df.loc[sub_indx,['x1','y1','j1','j2']].values

    #dc/(dndt)
    sub_indx = sub_df.index[(sub_df.species1==c_i)&(sub_df.species2==n_i)]
    dcdndt_indices = sub_df.loc[sub_indx,['x1','y1','j1','j2']].values

    #dn/(dndt)
    sub_indx = sub_df.index[(sub_df.species1==n_i)&(sub_df.species2==n_i)]
    dndndt_indices = sub_df.loc[sub_indx,['x1','y1','j1','j2']].values

    #dn/(dcdt)
    sub_indx = sub_df.index[(sub_df.species1==n_i)&(sub_df.species2==c_i)]
    dndcdt_indices = sub_df.loc[sub_indx,['x1','y1','j1','j2']].values

    #ds/(dcdt)
    sub_indx = sub_df.index[(sub_df.species1==s_i)&(sub_df.species2==c_i)]
    dsdcdt_indices = sub_df.loc[sub_indx,['x1','y1','j1','j2']].values

    #ds/(dndt)
    sub_indx = sub_df.index[(sub_df.species1==s_i)&(sub_df.species2==n_i)]
    dsdndt_indices = sub_df.loc[sub_indx,['x1','y1','j1','j2']].values

    #ds/(dsdt)
    sub_indx = sub_df.index[(sub_df.species1==s_i)&(sub_df.species2==s_i)]
    dsdsdt_indices = sub_df.loc[sub_indx,['x1','y1','j1','j2']].values

#     @numba.jit(cache=True,nopython=True)
    def calc_jac(t, y, dif_jac, rxn_jac, dcdcdt_indices, dcdndt_indices, 
                 dndndt_indices, dndcdt_indices, dsdcdt_indices, dsdndt_indices, dsdsdt_indices):
        rxn_jac[:,:] = dif_jac[:,:]
        #dc/(dcdt)
        for x1,y1,j1,j2 in dcdcdt_indices:
            rxn_jac[j1,j2] += rc*hill(y[n_i,y1,x1], Hn, Kn)

        #dc/(dndt)
        for x1,y1,j1,j2 in dcdndt_indices:
            rxn_jac[j1,j2] += rc*dhillda(y[n_i,y1,x1], Hn, Kn)*y[c_i,x1,y1]

        #dn/(dndt)
        for x1,y1,j1,j2 in dndndt_indices:
            rxn_jac[j1,j2] += -pn*dhillda(y[n_i,y1,x1], Hn, Kn)*y[c_i,x1,y1]

        #dn/(dcdt)
        for x1,y1,j1,j2 in dndcdt_indices:
            rxn_jac[j1,j2] += -pn*hill(y[n_i,y1,x1], Hn, Kn)

        #ds/(dcdt)
        for x1,y1,j1,j2 in dsdcdt_indices:
            rxn_jac[j1,j2] += xs*(y[c_i,y1,x1]>col_thresh)

        #ds/(dndt)
        for x1,y1,j1,j2 in dsdndt_indices:
            rxn_jac[j1,j2] += -rc*y[s_i,y1,x1]*dhillda(y[n_i,y1,x1],Hn,Kn)

        #ds/(dsdt)
        for x1,y1,j1,j2 in dsdsdt_indices:
            rxn_jac[j1,j2] += -ps

#         return rxn_jac
    
#     @numba.jit(cache=True)
    def calc_jac_wrapper(t,y):
        y.shape = (species,n_h,n_w)
        calc_jac(t,y,dif_jac, rxn_jac,dcdcdt_indices, dcdndt_indices, 
                 dndndt_indices, dndcdt_indices, dsdcdt_indices, dsdndt_indices, dsdsdt_indices)
        y.shape = species*n_w*n_h
        return rxn_jac
                             
                             
    @numba.jit(cache=True,nopython=True)
    def calc_f(y, d_y, diff_terms, nut_avail, p0):
        dx, Dc, Dn, rc, Kn, Hn, pn, xs, ps, leak, od = p0
        calc_diffusion(y, diff_terms)

        # Nutrient term
        nut_avail[:,:] = hill(y[n_i,:,:], Hn, Kn)

        # Cell growth and diffusion
        d_y[c_i,:,:] = (dx)*Dc*diff_terms[c_i,:,:] + rc * nut_avail * y[c_i,:,:]

        # Nutrient consumption
        d_y[n_i,:,:] = (dx)*Dn*diff_terms[n_i,:,:] - pn * nut_avail * y[c_i,:,:]

        # Synthase production
        d_y[s_i,:,:] = (xs * np.greater(y[c_i,:,:],col_thresh) - rc * y[s_i,:,:]) * nut_avail - ps * y[s_i,:,:]
        
    return dims, p0, initial_array, tmax, atol, rtol, calc_f, calc_jac_wrapper
#     return wrapper(dims, p0, initial_array, tmax, atol, rtol, calc_f, calc_jac_wrapper)

def sim_pad(p0, scale, tmax, prep_fn):
    return wrapper(*sim_pad_prep(p0, scale, tmax, prep_fn))


In [3]:
# p0 = np.array([1e-5, 3e-1, 2e-2, 75, 2, 1, 1e-8, 8e-3, 5e2, 1], dtype=np.float32)
# dims, _, initial_array, tmax, _, _, calc_f, calc_jac = sim_pad(p0,np.power(2,3).astype(np.int), 10*60, prep_pad_0)
# # out = calc_f()
# # out = sim_pad(p0,np.power(2,3).astype(np.int), 10*60, prep_pad_0)


In [4]:
# scale = 4
# p0 = np.array([1e-5, 3e-1, 2e-2, 75, 2, 1, 1e-8, 8e-3, 5e2, 1], dtype=np.float32)
# Dc, Dn,    rc, Kn, Hn, pn, xs,   ps, leak, od0 = p0
# dx = np.power((scale/4.5),2)
# p1 = np.array([dx, Dc, Dn, rc, Kn, Hn, pn, xs, ps, leak, od0])
# species, n_h, n_w, scale = dims
# # plt.imshow(calc_jac(0,initial_array.flatten())==0)
# # plt.imshow(calc_jac(0,initial_array.flatten())==0)
# n_vars = n_w*n_h*species
# initial_vec = initial_array.flatten()
# args=(np.zeros((species, n_h, n_w), dtype=np.float32,order='C'), 
#       np.zeros((species, n_h, n_w), dtype=np.float32,order='C'), 
#       np.zeros((n_h, n_w), dtype=np.float32,order='C'), 
#       p1)
# d_y, diff_terms, nut_avail, _ = args
# calc_lam = lambda y : calc_f(y,*args)
# %timeit calc_jac(0,initial_vec)
# # %timeit calc_diff_jac()
# # %timeit calc_jac(0,initial_array.flatten())
# # plt.imshow(jac_arr==0)
# # plt.figure(figsize=(20,20))
# # calc_arr = calc_jac(0,initial_array.flatten())
# # plt.imshow(calc_arr==0)
# # plt.figure(figsize=(20,20))
# # plt.imshow(np.isclose(jac_arr, calc_arr, 0.001))

In [5]:
plt.close('all')
scale = np.power(2,3).astype(np.int)
tmax = 10*60

#Params :         Dc, Dn,    rc, Kn, Hn, pn, xs,   ps, leak, od0
p0 = np.array([1e-5, 3e-1, 2e-2, 75, 2, 1, 1e-8, 8e-3, 5e2, 1], dtype=np.float32)
# Define parameter values to sample
Dc, Dn,    rc, Kn, Hn, pn, xs,   ps, leak, od0 = p0
rc_vec = np.power(10,np.linspace(-3,-1,4))
n_rc = len(rc_vec)
dn_vec = np.power(10,np.linspace(-3,-1,4))
n_dn = len(dn_vec)
kn_vec = np.linspace(10,90,4)
n_kn = len(kn_vec)
pn_vec = np.linspace(1,20,4)
n_pn = len(pn_vec)
od0_vec = [0.1,1,10]
n_od0 = len(od0_vec)

# Create parameter data frame
columns = 'Dc' ,'Dn' ,'rc' ,'Kn' ,'Hn' ,'pn' ,'xs' ,'ps' ,'leak' ,'od0'
index = np.arange(n_rc*n_dn*n_kn*n_pn*n_od0)
p0_df = pd.DataFrame(columns=columns, index=index)

i = 0
for rc in rc_vec:
    for Dn in dn_vec:
        for Kn in kn_vec:
            for pn in pn_vec:
                for od0 in od0_vec:
                    p0_df.iloc[i,:] = Dc, Dn,    rc, Kn, Hn, pn, xs,   ps, leak, od0
                    i += 1


# for scale_ind in np.arange(5,8):
#     for solver in ['RK45','LSODA']:
#         scale = np.power(2,scale_ind).astype(np.int)
#         print('scale:{} solver:{} pad0'.format(scale, solver))
#         %time sim_pad(p0_df.loc[0,:].values, scale, tmax, prep_pad_0)

# pad_fn_list = [prep_pad_0,prep_pad_1,prep_pad_2,prep_pad_3,prep_pad_4,prep_pad_5,prep_pad_6,prep_pad_7,prep_pad_8]
# for scale_ind in [3]:
#     for i, pad_fn in enumerate(pad_fn_list):
#         scale = np.power(2,scale_ind).astype(np.int)
#         print(scale, i)
#         %time sim_pad(p0_df.iloc[1,:].values, scale, tmax, pad_fn)
        
# def worker(p0_df):
#     pad_fns = [prep_pad_0,prep_pad_1,prep_pad_2,prep_pad_3,prep_pad_4,prep_pad_5,prep_pad_6,prep_pad_7,prep_pad_8]
#     for p0_ind in p0_df.index:
#         p0 = p0_df.loc[p0_ind,:].values
#         for pad_ind, pad_fn in enumerate(pad_fns):
#             out = sim_pad(p0, scale, tmax, pad_fn)
#             np.save('worker_outputs/pad{}_p{}_arr.npy'.format(pad_ind, p0_ind), out[0])
#             np.save('worker_outputs/pad{}_p{}_tvc.npy'.format(pad_ind, p0_ind), out[1])
#             anim = write_movie(out[0], out[1])
#             anim.save('worker_outputs//animation_pad{}_p{}.gif'.format(pad_ind,p0_ind), writer='pillow')    
    
# jobs = []
# n_proc = 4
# for i in range(n_proc):
#     p = Process(target=worker, args=(p0_df[i::n_proc].copy(),))
#     jobs.append(p)
#     p.start()
# p.join()

# plt.figure(figsize=(20,20))
# plt.imshow(prep_pad_4(scale))


In [ ]:
1

In [ ]:
c_i, n_i, s_i = np.arange(3)
def write_movie(im_arr, t_vec, skip=1, n_frames=200):
    
    frames, s, h, w = im_arr.shape
    t_points = np.arange(0,t_vec.max(),n_frames)
    f_points = np.arange(frames)
    
    #frames = len(t)
    
    t, s, h, w = im_arr.shape
    xticks = []
    xticklabels = []
    # First set up the figure, the axis, and the plot element we want to animate
    blank_array = np.zeros([h, w])
    fig, axs = plt.subplots(1,3, figsize=(10,7))
    im_list = [0,0,0]
    
    # Plot cell densities
    ax = axs[0]
    indxs = [c_i]
    vmax = im_arr[-1,indxs,:,:].sum(axis=0).max()
    vmin = im_arr[-1,indxs,:,:].sum(axis=0).min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none', aspect=1)
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks([])
    ax.set_xticklabels(xticklabels)
    ax.set_yticks([])
    ax.set_title('cell densities')
    im_list[0] = im
    
    # Plot nutrient densities
    ax = axs[1]
    indxs = [n_i]
    vmax = im_arr[:,indxs[0],:,:].max()
    vmin = im_arr[:,indxs[0],:,:].min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none', aspect=1)
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels)
    ax.set_yticks([])
    ax.set_title('nutrient')
    im_list[1] = im
    
    # Plot synthase densities
    ax = axs[2]
    indxs = [s_i]
    v_arr = im_arr[:,indxs,:,w//2:].prod(axis=1)
    vmax = v_arr.max()
    vmin = v_arr.min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none', aspect=1)
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels)
    ax.set_yticks([])
    ax.set_title('synthases')
    im_list[2] = im

    # animation function.  This is called sequentially
    t_points = np.linspace(0,t_vec.max(), 200)
    f_inds = []
    t_ind = 0
    for tp in t_points:
        while tp > t_vec[t_ind]:
            t_ind += 1
        f_inds.append(t_ind-1)
        
    def animate(t_point):
        i = f_inds[t_point]
        
        # Plot cell densities
        ax = axs[0]
        indxs = [c_i]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[0].set_array(frame_arr)

        # Plot nutrient densities
        ax = axs[1]
        indxs = [n_i]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[1].set_array(frame_arr)

        # Plot synthase densities
        ax = axs[2]
        indxs = [s_i]
        frame_arr = im_arr[i,indxs,:,:].prod(axis=0)
        im_list[2].set_array(frame_arr)

        #return im_list,

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = anm.FuncAnimation(fig, animate, interval=50, frames=n_frames)

#     anim.save('./animation_test.gif', writer='pillow')
    fig.tight_layout()
    plt.close('all')
    return anim
#     HTML(anim.to_html5_video())


In [12]:
c_i, n_i, s_i = np.arange(3)
scale = np.power(2,4).astype(np.int)
tmax = 1
# t_points = np.arange(0,tmax,0.000005)

#Params :         Dc, Dn,    rc, Kn, Hn, pn, xs,   ps, leak, od0
p0 = np.array([1e-5, 3e-1, 2e-2, 75, 2, 1, 1e-8, 8e-3, 5e2, 1], dtype=np.float32)

# out = sim_pad_prep(p0, scale, 1000, prep_pad_0)
# dims, p0, initial_array, tmax, atol, rtol, calc_f = out
dims, p0, initial_array, tmax, atol, rtol, calc_f, calc_jac_wrapper = out

y = initial_array.copy()
d_y = initial_array.copy()
diff_terms = np.zeros(y.shape, dtype=np.float32)
nut_avail = np.zeros(y.shape[1:], dtype=np.float32)
%timeit calc_f(y, d_y, diff_terms, nut_avail, p0)
%timeit f_ivp(0, y, d_y, diff_terms, nut_avail, p0, dims, calc_f)
%timeit calc_jac_wrapper(0, y)


67.2 µs ± 756 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
73.2 µs ± 964 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
56 ms ± 888 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
